# Statistics

In [ ]:
%%capture
%run ./phase-space.ipynb

```{autolink-concat}
```

In [ ]:
# pyright: reportUndefinedVariable=false
from __future__ import annotations

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from tensorwaves.function.sympy import create_parametrized_function
from tqdm.notebook import tqdm

from polarization import formulate_polarization
from polarization.io import perform_cached_doit

## Parameter loading

In [ ]:
n_bootstraps = 200

In [ ]:
rng = np.random.default_rng(seed=0)


def create_gaussian_distribution(
    mean: complex | float, std: complex | float, size: int
):
    if isinstance(mean, complex) and isinstance(std, complex):
        return (
            rng.normal(mean.real, std.real, size)
            + rng.normal(mean.imag, std.imag, size) * 1j
        )
    if isinstance(mean, (float, int)) and isinstance(std, (float, int)):
        return rng.normal(mean, std, size)
    raise NotImplementedError(
        f"Cannot create a Gaussian distribution for a mean/std type of {type(mean)},"
        f" {type(std)}"
    )


def smear_gaussian(
    parameter_values: dict[str, complex | float],
    parameter_uncertainties: dict[str, complex | float],
    size: int,
) -> dict[str, np.ndarray]:
    value_distributions = {}
    for k, mean in parameter_values.items():
        std = parameter_uncertainties[k]
        distribution = create_gaussian_distribution(mean, std, size)
        value_distributions[k] = distribution
    return value_distributions


# fmt:off
allowed_model_titles = [
    "Default amplitude model.",
    "Alternative amplitude model with K^*(892) with free mass and width.",
    "Alternative amplitude model with Lz(1670) with free mass and width.",
    "Alternative amplitude model with Lz(1690) with free mass and width.",
    "Alternative amplitude model with Deltares^{++}(1232) with free mass and width.",
    "Alternative amplitude model with Lz(1600), Deltares(1600)^{++}, Deltares(1700)^{++} with free mass and width.",
    "Alternative amplitude model with free Lz(1405) Flatt'e widths, indicated as G1 (pK channel) and G2 (Sigmapi).",
    "Alternative amplitude model with Lz(1800) contribution added with mass and width from Ref.~cite{PDG2020}.",
    "Alternative amplitude model with  Lz(1810) contribution added with mass and width from Ref.~cite{PDG2020}.",
    "Alternative amplitude model with Deltares(1620)^{++} contribution added with free mass and width.",
    "Alternative amplitude model in which a Relativistic Breit-Wigner is used for the K^*(700) contribution.",
    "Alternative amplitude model with K^*(700) with free mass and width.",
    "Alternative amplitude model with  K^*(1410) contribution added with mass and width from Ref.~cite{PDG2020}.",
    "Alternative amplitude model in which a Relativistic Breit-Wigner is used for the K^*(1430) contribution.",
    "Alternative amplitude model with K^*(1430) with free width.",
    "Alternative amplitude model with an additional overall exponential form factor exp(-alpha q^2) multiplying Bugg lineshapes. The exponential parameter is indicated as ``alpha''.",
    "Alternative amplitude model with free radial parameter d for the Lc resonance, indicated as dLc.",
    "Alternative amplitude model obtained using LS couplings.",
]
# fmt:on
model_parameters: dict[str, dict[str, np.ndarray]] = {}
for title in allowed_model_titles:
    symbol_parameters = load_model_parameters(
        "../data/modelparameters.json", decay, title, typ="value"
    )
    symbol_uncertainties = load_model_parameters(
        "../data/modelparameters.json", decay, title, typ="uncertainty"
    )
    values = {str(k): v for k, v in symbol_parameters.items()}
    uncertainties = {str(k): v for k, v in symbol_uncertainties.items()}
    model_parameters[title] = smear_gaussian(values, uncertainties, size=n_bootstraps)

## Function creation

In [ ]:
%%time
polarization_exprs = formulate_polarization(amplitude_builder)
unfolded_polarization_exprs = [
    perform_cached_doit(expr.doit().xreplace(model.amplitudes))
    for expr in tqdm(polarization_exprs, desc="Unfolding polarization expressions")
]
unfolded_intensity_expr = perform_cached_doit(model.full_expression)

This time, we do not {ref}`substitute certain parameters with their parameter defaults<polarization:Definition of free parameters>`, but lambdify the full expression, so that parameter values can be set for different models. Note that this makes lambdification slower.

In [ ]:
%%time
polarization_funcs = [
    create_parametrized_function(
        unfolded_polarization_exprs[xyz],
        parameters=model.parameter_defaults,
        backend="jax",
    )
    for xyz in tqdm(range(3))
]
intensity_func = create_parametrized_function(
    unfolded_intensity_expr,
    parameters=model.parameter_defaults,
    backend="jax",
)

## Statistical uncertainties

In [ ]:
X, Y, data, phsp_filter = generate_uniform_phsp(resolution=200)

In [ ]:
model_number = 0
model_pars = list(model_parameters.values())[model_number]
original_parameters = dict(intensity_func.parameters)
polarization_values = []
for i in tqdm(
    range(n_bootstraps), desc="Computing polarizations for parameter combinations"
):
    new_parameters = {k: v[i] for k, v in model_pars.items()}
    for func in polarization_funcs:
        func.update_parameters(original_parameters)
        func.update_parameters(new_parameters)
    polarization_values.append([func(data).real for func in polarization_funcs])
polarization_values = jnp.array(polarization_values)

In [ ]:
%config InlineBackend.figure_formats = ['png']

In [ ]:
fig, axes = plt.subplots(
    ncols=3,
    nrows=2,
    figsize=(15, 10),
    gridspec_kw={"width_ratios": [1, 1, 1.2]},
    sharex=True,
    sharey=True,
    tight_layout=True,
)
s1_label = R"$\sigma_1=m^2\left(K\pi\right)$"
s2_label = R"$\sigma_2=m^2\left(pK\right)$"
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)
Z_mean = jnp.mean(polarization_values, axis=0)
Z_std = jnp.std(polarization_values, axis=0)
Z_std_max = jnp.nanmax(Z_std)
for i in range(3):
    axes[0, i].set_title(Rf"$\alpha_{'xyz'[i]}$")
    mesh = axes[0, i].pcolormesh(X, Y, Z_mean[i], cmap=cm.coolwarm)
    mesh.set_clim(vmin=-1, vmax=+1)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha$ value averaged with {n_bootstraps} bootstraps")
    mesh = axes[1, i].pcolormesh(X, Y, Z_std[i], cmap=cm.Reds)
    mesh.set_clim(vmin=0, vmax=+Z_std_max)
    axes[1, i].set_xlabel(s1_label)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation")
plt.show()

## Systematic uncertainties